# Imaginary Time Evolution
In this tutorial we show how to use the FTPS solver to compute imaginary time Green functions (GF). Since FTPS uses time evolution, it is literally as simple as replacing $it$ by $\tau$ to do that. Unfortunately there is another issue with the current (at the time of writing of this tutorial) TRIQS version. FTPS is a $T=0$ method, but TRIQS currently has no $T=0$ imaginary time mesh implemented. We will discuss why this matters and how to work around that to obtain the Matsubara Green function.
As in the real-time case, FTPS actually calculates the greater- and lesser Green's function in imaginary time:

$$G^>(\tau) = \langle 0 | c(\tau) c^\dagger | 0 \rangle = \langle 0 | c e^{-\tau(H-E_0)} c^\dagger | 0 \rangle $$
   
$$G^<(\tau) = \langle 0 | c^\dagger c(\tau) | 0 \rangle = \langle 0 | c^\dagger e^{\tau(H-E_0)} c | 0 \rangle. $$

Since $H-E_0 > 0$, the greater GF has to be calculated only for $\tau > 0$, while the lesser one only for $\tau<0$. Otherwise we would have GFs increasing exponentially in time with an ill-defined $\tau \to \infty$ limit. As we will see below, this is exactly what is needed for the Matsubara GF.

# Differences at $\beta = \infty$
The problem with the usual $\tau$-grid is that for $\beta \to \infty$ we loose the values of the Green's function at $\beta$ which encodes roughly speaking half of the necessary information as we show in the following. The definition of the Matsubara GF is:

$$G^{Mat} = -T_\tau \langle c(\tau) c^\dagger \rangle = -\underbrace{\langle c(\tau) c^\dagger \rangle}_{G^>(\tau)} \cdot \Theta(\tau>0) +  \underbrace{ \langle c^\dagger c(\tau) \rangle}_{G^<(\tau)} \cdot \Theta(\tau<0)$$

One has to keep in mind that $G^{Mat}$ is anti-periodic with period $\beta$ and that greater and lesser Green's functions decay exponentially in $\tau$. So if we think of the usual $\tau$-grid $[0,\beta]$, $G^{Mat}$ is for small $\tau$ $0 \leq \tau \ll \beta/2$ equal to $G^>(\tau)$ and for large $\beta/2 \ll \tau \leq \beta$ equal to $G^<(\tau)$. This distinction only makes sense at large values of $\beta$ because otherwise the greater and lesser components overlap. <br>
If we would take the limit $\beta \to \infty$ on the usual $\tau$-grid, we would end up with $G^{>}(\tau)$ alone and would loose all the information about $G^{<}(\tau)$. Hence we have to fall back to a different $\tau$-grid using the interval $[-\beta/2, \beta/2 ]$. This is much closer to the definition of $G^{Mat}$ given above and is the appropriate choice for $T=0$ i.e. $\beta \to \infty$. Calculating the lesser and greater GF for these values of $\tau$ is also numerically stable, since both time evolutions are performed with a negative total exponent meaning that both decay exponentially. <br><br>
Unfortunately TRIQS currently has no such $\tau$-grid implemented and that is why we have to work around it. The FTPS solver therefore uses the usual <code>S.G_gr</code> and <code>S.G_le</code> members to store the imaginary time GF although these are actually TRIQS real-time GFs. This implies that the solver cannot calculate real- and imaginary GFs at the same time. The decision on which one should be computed is made in the parameters of the <code>S.solve()</code>-function as we will see in the following example.

In [ ]:
# some imports
#%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np

import forktps as ftps
from forktps.DiscreteBath import *
from forktps.Helpers import *

from triqs.gf import *

# as usual set up a simple model and lets switch off interactions to be able to compare to the exact solution
Norbs = 1
inds = list(range(Norbs))
gfstruct = [['up',inds], ['dn',inds]]

dup = GfReFreq(indices = inds, window = (-3., 3.), n_points = 3001, name = "up")
ddn = GfReFreq(indices = inds, window = (-3., 3.), n_points = 3001, name = "dn")

# to make it a little more interesting use different baths for up and down
for i in inds:
    dup[i,i] << SemiCircular(half_bandwidth = 1.3)
    ddn[i,i] << SemiCircular(half_bandwidth = 2.0)

Delta = BlockGf(name_list = ('up','dn'), block_list = (dup,ddn), make_copies = False)

Nb = 9 #number of bath sites
bath = DiscretizeBath(Delta=Delta, Nb=Nb) #bath object storing all relevant information for the bath

e0 = ftps.solver_core.Hloc(gfstruct) #give the local Hamiltonian the right block structure
e0.Fill( [[-0.5]] ) # e0 is a matrix hence the double brakets

# Interaction parameters U, Hund's Coupling J and inter-orbital U'. 
Hint = ftps.solver_core.HInt(u=0.,j=0., up=0., dd = True)

In [ ]:
S = ftps.Solver(gf_struct = gfstruct , nw = 3001, wmin=-3., wmax=3.)
# TRIQS solver often have the Weiss field as member providing information about the bath. 
# The bath and e0 carry the exact same information for the ForkTPS solver.
S.b = bath
S.e0 = e0

# Using the argument imag_tevo in tevoParams, we make the choice to calculate the imaginary time Green's function. (default value of imag_tevo is False)
tevoParams = ftps.solver.TevoParams(dt = 0.5, time_steps=40, 
                                    #here the magic happens
                                    imag_tevo = True )

S.solve( h_int = Hint, tevo = tevoParams )

First, lets look at the actual solver output. Since we already abuse real-time GFs for imaginary time, the lesser component is abused a little more and stores $\tau<0$ values into a TRIQS GF living on a positive $t$-grid with the convention $\tau \to -\tau$ for $G^<(\tau)$.
<br><br>
Nevertheless, the user does not need to worry about these conventions, because <code>Helpers.py</code> provides a function <code>MakeMatsubara(Ggr, Gle)</code> that computes the correct Matsubara GF from a greater and a lesser GF as given by the solver. One thing to note here is that TRIQS meshes cannot deal with the discontinuity at $\tau=0$. Therefore the <code>MakeMatsubara(Ggr, Gle)</code> function sets the value at $\tau=0$ to the average $0.5(G^>(0)+G^<(0))$ which is correct for integration with the trapezoid rule. Still, be aware that this happens and check that this choice does not alter your results.

In [ ]:
tauMesh = MeshImTime(20, 'Fermion', 100)
Ggr_exact = CalcGgr(bath, e0, mesh = tauMesh ) # by giving CalcGgr an imaginary time mesh, it computes the G>(tau)
Gle_exact = CalcGle(bath, e0, mesh = tauMesh ) # by giving CalcGgr an imaginary time mesh, it computes the G<(tau)

# this function returns the correct Matsubara GF for G> and G< as returned by FTPS.
# it allows the user to not worry too much about how the convention works exactly
Gmat = MakeMatsubara(S.G_gr, S.G_le)


# rest of the code is just plotting the results
plt.figure(figsize = (8,4))
plt.subplot(1,2,1)
plt.title(R"$ G^>(\tau) $" )
for [name, g], [_,gex] in zip(GFDiag(S.G_gr), GFDiag(Ggr_exact) ):
    plt.plot( getX(tauMesh), gex.real.data, label = f'Exact {name}')
    plt.plot( getX(g.mesh), g.real.data,    label = f'Solver {name}', marker = 'o', ls = '--', markevery = 3 )  
plt.xlabel(R"$\tau$")
plt.legend()
         
plt.subplot(1,2,2)
plt.title(R"$ G^<(\tau) $" )
for [name, g],[_,gex] in zip(GFDiag(S.G_le), GFDiag(Gle_exact) ):
    plt.plot( getX(tauMesh), gex.real.data, label = f'Exact {name}')
    plt.plot( getX(g.mesh), g.real.data,    label = f'Solver {name}', marker = 'o', ls = '--', markevery = 3 )
plt.xlabel(R"$\tau$")
plt.legend()
        
    
plt.figure(figsize = (8,4))
plt.title('Matsubara Green function')
for name, g in GFDiag(Gmat):
    plt.plot( getX(g.mesh), g.real.data, 'o-', label = R'$G^{Mat}_{%s}(\tau)$ '%name)
    plt.ylabel(R"$G(\tau)$")
plt.xlim(-20,20)
plt.xlabel(R'$\tau$')
plt.legend()
plt.annotate( R'$\tau = 0$ is average'+'\n$0.5(G^>(0)+G^<(0))$\nIn truth there would be a discontinuity', (0.2, 0.165), xytext = (2.5, 0.16), arrowprops=dict(arrowstyle="->") )

To get the GF for imaginary Frequencies, we can just Fourier Transform <code>Gmat</code>. This can be done using the appropriate imaginary frequency mesh or also a real-frequency mesh because formally at $\beta=\infty$, the Matsubara frequencies become dense and it starts to make sense to evaluate $G(i\omega)$ on arbitrary frequency points. Note however that the solver provides the GF of a discretized model and hence has limited frequency resolution no matter what. <br><br>

In the current toy-model the finite resolution shows itself in that $G(i\omega)$ bends over towards zero at $i\omega \approx 0.25$ (for $N_b=9$). If one would increase the bath size, this would move towards lower frequencies.

#### Some Notes
1. The tiny oscillations come from the finite maximal time $\tau$. 
2. The finite time-step $\Delta \tau$ used in the time evolution becomes a maximal frequency and this affects $G(i\omega)$ such that it becomes periodic making the very high-frequency GF deviate from the exact result. This is purely a Fourier Transform issue and if one wants to get the high-frequency tails, one should think about interpolating $G(\tau)$.

In [ ]:
iwMesh = IWMesh(200, 150) # create an iw-Mesh for beta = 200 using 500 matsubaras
wMesh = WMesh(-5,5,1000) # create an arbitrary real-freuqency mesh

Giw_iwMesh = FourierTransform(Gmat, iwMesh, eta = 0.) # Fourier Transform without broadening!
Giw_wMesh  = FourierTransform(Gmat,  wMesh, eta = 0.) # Fourier Transform without broadening!

plt.figure()
plt.title('$G(i\omega)$ using real-frequency and imaginary-frequency grid.')
for [name, giw],[_, gw] in zip( GFDiag(Giw_iwMesh), GFDiag(Giw_wMesh) ):
    plt.plot( getX(giw.mesh), giw.imag.data, 'o-' ,label = f'Spin {name} $i\omega$-Mesh' )
    plt.plot( getX(gw.mesh), gw.imag.data,label = f'Spin {name} $\omega$-Mesh' )
plt.legend()